# Auxilary stuff to load trained gaussians

In [3]:
from arguments import ModelParams, PipelineParams, OptimizationParams
from argparse import ArgumentParser, Namespace
import matplotlib.pyplot as plt
import numpy as np
from scene import Scene, GaussianModel
import torch
from gaussian_renderer import render
import os
import imageio
from tqdm import tqdm
import PIL
import PIL.ImageDraw
from diff_gaussian_rasterization import split_gaussians

In [9]:
!ls /data/output/gaussian-splatting/bonn_person_static_inpainted

bonn_person_static_inpainted		    bonn_person_static_wider
bonn_person_static_undistorted_masks	    bonn_person_static_wider_10
bonn_person_static_undistorted_masks_green  bonn_person_static_wider_15


In [10]:
exp_name = "bonn_person_static_inpainted"

parser = ArgumentParser(description="Training script parameters")
lp = ModelParams(parser)
pipeline = PipelineParams(parser)
args = Namespace(
    sh_degree = 3,
    images = "images",
    resolution = -1,
    white_background = False,
    data_device = "cuda",
    eval = False,
    source_path = "/data/bonn_person_inpainted",
    model_path = os.path.join("/data/output/gaussian-splatting/", exp_name),
    masked = False
)

dataset = lp.extract(args)

In [11]:
gaussians = GaussianModel(dataset.sh_degree)
scene = Scene(dataset, gaussians, load_iteration=30_000)
cams = sorted(scene.getTrainCameras(), key= lambda x: int(x.image_name))

Loading trained model at iteration 30000
Reading camera 580/580
Loading Training Cameras
Loading Test Cameras


# Splitting visualization

In [12]:
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

# Sample points and convert torch tensor to numpy array for visualization
def sample_gaussian(mean, cov, n_points=1000):
    return np.random.multivariate_normal(mean.detach().cpu().numpy(), 
                                         cov.detach().cpu().numpy(), 
                                         n_points)

# Define the separating plane
def plane(x, y, n, d):
    # z = ax + by + c => c = -(ax + by + d)/c to get z, use substitute values 
    return (-n[0] * x - n[1] * y - d) / n[2]

def plot_gaussians(n, d, m_l, cov_l, o_l, m_r, cov_r, o_r, m_0, cov_0, o_0):
    # Sampling points
    points_original = sample_gaussian(m_0, cov_0)
    points_l = sample_gaussian(m_l, cov_l)
    points_r = sample_gaussian(m_r, cov_r)

    n_detached = n.detach().cpu().numpy()
    d_detached = float(d)

    # Generate grid for plane
    x = np.linspace(np.min([points_original[:, 0], points_l[:, 0], points_r[:, 0]]), 
                    np.max([points_original[:, 0], points_l[:, 0], points_r[:, 0]]), 20)
    y = np.linspace(np.min([points_original[:, 1], points_l[:, 1], points_r[:, 1]]), 
                    np.max([points_original[:, 1], points_l[:, 1], points_r[:, 1]]), 20)
    X, Y = np.meshgrid(x, y)
    
    Z = plane(X, Y, n_detached, d_detached)

    # Creating the 3D plot
    fig = go.Figure()

    # Adding scatter plots for the Gaussian samples
    fig.add_trace(go.Scatter3d(x=points_original[:, 0], y=points_original[:, 1], z=points_original[:, 2],
                            mode='markers', marker=dict(color='blue', opacity=o_0.item()),
                            name='Original Gaussian'))

    fig.add_trace(go.Scatter3d(x=points_l[:, 0], y=points_l[:, 1], z=points_l[:, 2],
                            mode='markers', marker=dict(color='red', opacity=o_l.item()),
                            name='Left Gaussian'))

    fig.add_trace(go.Scatter3d(x=points_r[:, 0], y=points_r[:, 1], z=points_r[:, 2],
                            mode='markers', marker=dict(color='green', opacity=o_r.item()),
                            name='Right Gaussian'))

    # Adding the plane
    fig.add_trace(go.Surface(x=X, y=Y, z=Z, opacity=0.5,
                            showscale=False, name='Separating Plane'))

    # Updating layout for better visualization
    fig.update_layout(scene=dict(xaxis_title='X Axis', yaxis_title='Y Axis', zaxis_title='Z Axis'),
                    title='3D Gaussian Distributions and Separating Plane')

    fig.show()   

def split_gaussian_by_half(opacity_0, means_0, cov_0, n):
    C = 0.5
    D = 1 / np.sqrt(2 * torch.pi)

    L_0 = torch.bmm(cov_0, n.permute(0, 2, 1))
    tao = torch.sqrt(torch.bmm(n, L_0))
    L_0_T = L_0.permute(0, 2, 1)

    offset = ((L_0 * D) / (tao * C)).squeeze()
    means_l = means_0 - offset
    means_r = means_0 + offset

    cov = cov_0 - (torch.bmm(L_0, L_0_T) / tao**2) * (D**2 / C**2)
    opacity = opacity_0 * C

    return means_l, means_r, opacity, cov

def get_inhomogenity_mask(scaling, gamma_thresh = 5):
    top_2 = torch.topk(scaling, 2, dim=1).values
    gammas = top_2[:,0] / top_2[:,1]
    return gammas > gamma_thresh


def batch_symmetric_matrix(batch):
    a, b, c, d, e, f = batch[:, 0], batch[:, 1], batch[:, 2], batch[:, 3], batch[:, 4], batch[:, 5]
    matrices = torch.stack([torch.stack([a, b, c], dim=1),
                            torch.stack([b, d, e], dim=1),
                            torch.stack([c, e, f], dim=1)], dim=1)
    return matrices

def build_rotation(r):
    norm = torch.sqrt(r[:,0]*r[:,0] + r[:,1]*r[:,1] + r[:,2]*r[:,2] + r[:,3]*r[:,3])

    q = r / norm[:, None]

    R = torch.zeros((q.size(0), 3, 3), device='cuda')

    r = q[:, 0]
    x = q[:, 1]
    y = q[:, 2]
    z = q[:, 3]

    R[:, 0, 0] = 1 - 2 * (y*y + z*z)
    R[:, 0, 1] = 2 * (x*y - r*z)
    R[:, 0, 2] = 2 * (x*z + r*y)
    R[:, 1, 0] = 2 * (x*y + r*z)
    R[:, 1, 1] = 1 - 2 * (x*x + z*z)
    R[:, 1, 2] = 2 * (y*z - r*x)
    R[:, 2, 0] = 2 * (x*z - r*y)
    R[:, 2, 1] = 2 * (y*z + r*x)
    R[:, 2, 2] = 1 - 2 * (x*x + y*y)
    return R



In [13]:
means3D, rotations, scales, opacity = gaussians.get_xyz, gaussians.get_rotation, gaussians.get_scaling, gaussians.get_opacity

mask = get_inhomogenity_mask(scales**2, 5)
m_0, r_0, s_0, o_0 = means3D[mask], rotations[mask], scales[mask], opacity[mask]
R_0 = build_rotation(r_0)
cov_0 = batch_symmetric_matrix(gaussians.get_covariance()[mask]).cuda()

max_index = torch.argmax(s_0, dim=1)
n = R_0[torch.arange(max_index.shape[0]).unsqueeze(1), :, max_index.unsqueeze(1)]
d = -torch.bmm(n, m_0.unsqueeze(2))

In [14]:
means_l, means_r, opacity_res, cov_res = split_gaussians(m_0, o_0, cov_0, n)

In [18]:
idx = 1020
plot_gaussians(n.squeeze()[idx], d.squeeze()[idx], means_l[idx], cov_res[idx], opacity_res[idx], means_r[idx], cov_res[idx], opacity_res[idx], m_0[idx], cov_0[idx], o_0[idx])

# time comparsion

In [19]:
%%timeit
split_gaussians(m_0, o_0, cov_0, n)

60.3 µs ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
%%timeit
split_gaussian_by_half(o_0, m_0, cov_0, n)

538 µs ± 660 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
